---

# CSCI 3202, Fall 2022
# Homework 4
# Due: Friday September 30, 2022 at 6:00 PM

<br> 

### Your name: Giuliano Costa

<br> 

---

Some useful packages and libraries:




In [2]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter


---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>

$$\frac{(1+number of heads)}{(2+number of flips)}$$

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [127]:
## Your code goes here.
typos = pd.read_csv("./data/typos20.data", sep=' ', header = None, names=["State", "Output"])
array = typos.to_dict()

letters = ['_', 'a', 'b', 'c', 'd', 'e', 'g', 'f', 'h', 'j', 'k', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#totals = sum(map((0).__eq__, array.values()))

typos
## Your code needs to display the first 10 lines of your emission and transition tables
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname

# Your code also needs to be structured as an HMM class

#Emissions:{'_': 669, 'a': 0, 'b': 222, 'c': 386, 'd': 280, 'e': 1, 'f': 41, 'g': 199, 'h': 4, 'i': 270, 'j': 22, 'k': 190, 'l': 1345, 'm': 191, 'n': 1924, 'o': 3, 'p': 179, 'q': 2, 'r': 892, 's': 640, 't': 1651, 'u': 159, 'v': 327, 'w': 27, 'x': 4, 'y': 246, 'z': 6} 
#Counts: 'a': 7873, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0, 'q': 675, 'r': 0, 's': 681, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 651} 
#use a dictionary of dictionaries

,State,Output
0,b,b
1,y,y
2,_,_
3,t,y
4,h,j
...,...,...
160975,n,n
160976,t,t
160977,i,i
160978,o,o


In [128]:
states = typos['State'] #.str.count('a')
output = typos['Output']

state_sums = {}
output_sums = {}

for i in letters:
    state_sums[i] = states.str.count(i).sum()
    output_sums[i] = output.str.count(i).sum()

In [129]:
# Create a raw transition matrix
a = typos.to_numpy()
matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)

matrix = matrix.rename_axis("State", axis="columns")
matrix = matrix.rename_axis("Output", axis="rows")

# Normalize the rows according to the laplace smoothing.
matrix = matrix + 1
#transition_probs = matrix.divide(matrix.sum(axis=1) + 160980*2, axis=0)
transition_probs = matrix.divide(matrix.sum(axis=1), axis=0)

#     A     B     C    D
#A  0.0  0.50  0.00  0.5
#B  0.5  0.25  0.25  0.0
#C  0.0  1.00  0.00  0.0
#D  0.5  0.00  0.00  0.5
#transition_probs.rename_axis("State", axis="columns")

In [ ]:
#Transition

states = typos['State'] #.str.count('a')
#output = typos['Output']

state_sums = {}
total = 0
for i in letters:
    state_sums[i] = states.str.count(i).sum()
    total += state_sums[i]

a = {k: v / total for k, v in state_sums.items()}
a

In [130]:
states = typos['State'] #.str.count('a')
output = typos['Output']

state_sums = {}
total = 0
for i in letters:
    state_sums[i] = states.str.count(i).sum()
    total += state_sums[i]

a = {k: v / total for k, v in state_sums.items()}
a

{'_': 0.16885948565039136,
 'a': 0.0613740837371102,
 'b': 0.012094670145359672,
 'c': 0.028022114548391104,
 'd': 0.026493974406758602,
 'e': 0.1049757733879985,
 'g': 0.01583426512610262,
 'f': 0.01928189837246863,
 'h': 0.038924089949062,
 'j': 0.0006149832277301528,
 'k': 0.0034724810535470244,
 'i': 0.06334327245620575,
 'l': 0.037048080506895266,
 'm': 0.022008945210585166,
 'n': 0.056870418685551,
 'o': 0.06960491986582185,
 'p': 0.01890296931295813,
 'q': 0.0008883091067213319,
 'r': 0.048018387377313954,
 's': 0.056354826686544915,
 't': 0.08033917256802087,
 'u': 0.023064977015778358,
 'v': 0.011131817617095292,
 'w': 0.012920859734128463,
 'x': 0.0016151074667660578,
 'y': 0.017213318424649026,
 'z': 0.0007267983600447261}

In [152]:
class HMM():
    def __init__(self, df):
        
        #### Initial ####
        states = df['State'] #.str.count('a')
        state_sums = {} 
        total = 0
        
        for i in letters:
            state_sums[i] = states.str.count(i).sum()
            total += state_sums[i]
        #Sum state letter counts and divide by sum of letters
        a = {k: v / total for k, v in state_sums.items()}        
        init_distribution = a
        
        #### Transition ####
        #change from s to s'
        a = df.to_numpy
        matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)
        
        #### Emission ####
        a = df.to_numpy()
        matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)

        matrix = matrix.rename_axis("State", axis="rows")
        matrix = matrix.rename_axis("Output", axis="columns")

        # Normalize the rows according to the laplace smoothing.
        denom = matrix
        matrix = matrix + 1
        #transition_probs = matrix.divide(matrix.sum(axis=1) + 160980*2, axis=0)
        emission_probs = matrix.divide(denom.sum(axis=1) + 27, axis=0)
        self.em = emission_probs
        self.init = init_distribution
        
    def tables(self):
        return  self.em, self.init 


In [153]:
test = HMM(typos)

In [154]:
em, init = test.tables()

In [155]:
init

{'_': 0.16885948565039136,
 'a': 0.0613740837371102,
 'b': 0.012094670145359672,
 'c': 0.028022114548391104,
 'd': 0.026493974406758602,
 'e': 0.1049757733879985,
 'g': 0.01583426512610262,
 'f': 0.01928189837246863,
 'h': 0.038924089949062,
 'j': 0.0006149832277301528,
 'k': 0.0034724810535470244,
 'i': 0.06334327245620575,
 'l': 0.037048080506895266,
 'm': 0.022008945210585166,
 'n': 0.056870418685551,
 'o': 0.06960491986582185,
 'p': 0.01890296931295813,
 'q': 0.0008883091067213319,
 'r': 0.048018387377313954,
 's': 0.056354826686544915,
 't': 0.08033917256802087,
 'u': 0.023064977015778358,
 'v': 0.011131817617095292,
 'w': 0.012920859734128463,
 'x': 0.0016151074667660578,
 'y': 0.017213318424649026,
 'z': 0.0007267983600447261}

In [159]:
em

Output,_,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
State,,,,,,,,,,,,,,,,,,,,,
_,0.999044,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,...,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037
a,0.000101,0.794792,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,...,0.068235,0.000101,0.068840,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,0.065812
b,0.000507,0.000507,0.798886,0.000507,0.000507,0.000507,0.000507,0.046099,0.043566,0.000507,...,0.000507,0.000507,0.000507,0.000507,0.000507,0.047112,0.000507,0.000507,0.000507,0.000507
c,0.000220,0.000220,0.000220,0.793521,0.044954,0.000220,0.052887,0.000220,0.000220,0.000220,...,0.000220,0.000220,0.000220,0.000220,0.000220,0.053768,0.000220,0.050022,0.000220,0.000220
d,0.000233,0.000233,0.000233,0.041007,0.801025,0.040541,0.033784,0.000233,0.000233,0.000233,...,0.000233,0.000233,0.040541,0.000233,0.000233,0.000233,0.000233,0.038211,0.000233,0.000233
e,0.000059,0.000059,0.000059,0.000059,0.066348,0.800662,0.000059,0.000059,0.000059,0.000059,...,0.000059,0.064812,0.000059,0.000059,0.000059,0.000059,0.066820,0.000059,0.000059,0.000059
f,0.000319,0.000319,0.000319,0.033536,0.030981,0.000319,0.802619,0.036091,0.000319,0.000319,...,0.000319,0.029064,0.000319,0.032577,0.000319,0.028745,0.000319,0.000319,0.000319,0.000319
g,0.000388,0.000388,0.041925,0.000388,0.000388,0.000388,0.041537,0.800466,0.030668,0.000388,...,0.000388,0.000388,0.000388,0.041149,0.000388,0.036102,0.000388,0.000388,0.000388,0.000388
h,0.000159,0.000159,0.043699,0.000159,0.000159,0.000159,0.000159,0.039886,0.791038,0.000159,...,0.000159,0.000159,0.000159,0.000159,0.000159,0.000159,0.000159,0.000159,0.037979,0.000159



## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [ ]:
## Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
## You also need to display the error rate for your algorithm.

def Viterbi(HMM, ev):
    

## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 
Purpose should be clear. It doesn't need to be long, but it should be clear.
### Procedure:
##### Transition Probabilities
In this section explain the way you have generated the Transition probabilities and the data structure you used.
##### Emission Probabilities
In this section explain the way you have generated the Emission probabilities and the data structure you used.
##### Viterbi Algorithm
In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.
### Data
Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results
Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


Your report goes here.
